In [1]:
# LangChain Multi-Agent System
# Save as: notebooks/05_langchain_multi_agent_system.ipynb

# Cell 1: Setup and Imports
from langchain_ollama import OllamaLLM
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import time
from datetime import datetime
import asyncio
from typing import Dict, List, Any

# Initialize LLM
print("🔧 Initializing LangChain LLM for multi-agent workflows...")
llm = OllamaLLM(
    model="llama3.1:8b-instruct-q4_K_M",
    base_url="http://ollama:11434"
)
print("✅ LangChain LLM ready for multi-agent systems!")

🔧 Initializing LangChain LLM for multi-agent workflows...
✅ LangChain LLM ready for multi-agent systems!


In [2]:
# Cell 2: Define Agent Classes
class SpecializedAgent:
    """Base class for specialized agents"""
    
    def __init__(self, role: str, expertise: str, system_prompt: str, llm):
        self.role = role
        self.expertise = expertise
        self.system_prompt = system_prompt
        self.llm = llm
        self.memory = []  # Store previous interactions
        
    def analyze(self, topic: str, context: str = "", specific_instructions: str = "") -> Dict[str, Any]:
        """Perform analysis on the given topic"""
        
        # Create the full prompt
        full_prompt = f"""You are a {self.role}.

{self.system_prompt}

Topic to analyze: {topic}

{f"Context from previous analyses: {context}" if context else ""}

{f"Specific instructions: {specific_instructions}" if specific_instructions else ""}

Provide your expert analysis focusing on your area of expertise. Be detailed, specific, and practical."""

        start_time = time.time()
        
        try:
            response = self.llm.invoke(full_prompt)
            duration = time.time() - start_time
            
            # Store in memory
            analysis_result = {
                "agent": self.role,
                "topic": topic,
                "analysis": response,
                "duration": duration,
                "word_count": len(response.split()),
                "timestamp": datetime.now().isoformat()
            }
            
            self.memory.append(analysis_result)
            
            print(f"✅ {self.role} completed analysis in {duration:.1f}s ({len(response.split())} words)")
            return analysis_result
            
        except Exception as e:
            print(f"❌ {self.role} analysis failed: {e}")
            return {
                "agent": self.role,
                "topic": topic,
                "analysis": f"Analysis failed: {e}",
                "duration": time.time() - start_time,
                "word_count": 0,
                "error": str(e)
            }

In [3]:
# Cell 3: Create Specialized Agents
print("👥 Creating specialized LangChain agents...")

# Healthcare Domain Expert
healthcare_expert = SpecializedAgent(
    role="Healthcare Domain Expert",
    expertise="Medical technology, clinical workflows, healthcare regulations",
    system_prompt="""You are a healthcare industry veteran with 15+ years of experience in 
    medical technology adoption, clinical workflows, and healthcare regulations. You understand 
    how technology impacts patient care, hospital operations, and medical decision-making processes.
    
    Focus your analysis on:
    - Clinical workflow integration
    - Patient care impact
    - Healthcare provider adoption challenges
    - Regulatory compliance requirements
    - Real-world implementation in healthcare settings
    
    Provide insights that only someone with deep healthcare industry experience would know.""",
    llm=llm
)

# Technical Analyst
technical_analyst = SpecializedAgent(
    role="AI Technical Analyst",
    expertise="AI/ML systems, technical architecture, implementation challenges",
    system_prompt="""You are a senior AI engineer specializing in healthcare AI systems with 
    deep knowledge of machine learning model validation, data pipelines, and integration challenges.
    
    Focus your analysis on:
    - AI/ML model requirements and validation
    - Data infrastructure and pipeline needs
    - Scalability and performance considerations
    - Security and privacy technical requirements
    - Implementation complexity and technical risks
    - System architecture and integration challenges
    
    Provide technical insights with specific implementation details and engineering considerations.""",
    llm=llm
)

# Regulatory Specialist
regulatory_specialist = SpecializedAgent(
    role="Healthcare Regulatory Specialist",
    expertise="FDA regulations, HIPAA compliance, medical device approval",
    system_prompt="""You are a regulatory affairs expert with deep knowledge of FDA approval 
    processes, HIPAA compliance, international medical device regulations, and healthcare data privacy.
    
    Focus your analysis on:
    - FDA approval pathways and requirements
    - HIPAA and data privacy compliance
    - International regulatory considerations
    - Clinical trial requirements
    - Liability and legal risk factors
    - Regulatory roadmap and timeline
    
    Provide specific regulatory guidance and compliance strategies.""",
    llm=llm
)

# Economic Analyst
economic_analyst = SpecializedAgent(
    role="Healthcare Economics Analyst",
    expertise="Healthcare economics, ROI analysis, market dynamics",
    system_prompt="""You are a healthcare economist who analyzes the financial impact of new 
    technologies on healthcare systems, insurance models, and hospital budgets.
    
    Focus your analysis on:
    - Cost-benefit analysis for healthcare systems
    - ROI calculations for hospitals and providers
    - Impact on healthcare spending and insurance
    - Market size and growth projections
    - Economic barriers to adoption
    - Financial sustainability models
    
    Provide quantitative analysis with specific financial metrics and business case elements.""",
    llm=llm
)

# Strategic Synthesizer
strategic_synthesizer = SpecializedAgent(
    role="Strategic Content Synthesizer",
    expertise="Strategic analysis, cross-domain synthesis, executive communication",
    system_prompt="""You are an expert strategic analyst who excels at synthesizing complex 
    information from multiple domains into cohesive strategic recommendations.
    
    Your role is to:
    - Integrate insights from healthcare, technical, regulatory, and economic analyses
    - Identify key strategic themes and interconnections
    - Create actionable recommendations for different stakeholders
    - Develop implementation roadmaps with clear priorities
    - Assess risks and mitigation strategies
    - Communicate complex concepts to executive audiences
    
    Synthesize information into clear, strategic insights that drive decision-making.""",
    llm=llm
)

agents = [healthcare_expert, technical_analyst, regulatory_specialist, economic_analyst, strategic_synthesizer]
print(f"✅ Created {len(agents)} specialized agents!")

👥 Creating specialized LangChain agents...
✅ Created 5 specialized agents!


In [4]:
# Cell 4: Multi-Agent Workflow Orchestrator
class MultiAgentOrchestrator:
    """Orchestrates multi-agent workflows"""
    
    def __init__(self, agents: List[SpecializedAgent]):
        self.agents = agents
        self.workflow_history = []
        
    def run_sequential_analysis(self, topic: str, analysis_type: str = "comprehensive") -> Dict[str, Any]:
        """Run sequential analysis with all agents"""
        
        print(f"🚀 Starting {analysis_type} multi-agent analysis: {topic}")
        print("=" * 70)
        
        start_time = time.time()
        results = []
        context = ""
        
        # Phase 1: Individual Expert Analyses
        for i, agent in enumerate(self.agents[:-1]):  # All except synthesizer
            print(f"\n📋 Phase {i+1}: {agent.role}")
            
            # Customize instructions based on agent
            if agent.role == "Healthcare Domain Expert":
                instructions = "Focus on clinical impact and healthcare provider perspectives."
            elif agent.role == "AI Technical Analyst":
                instructions = "Focus on technical feasibility and implementation challenges."
            elif agent.role == "Healthcare Regulatory Specialist":
                instructions = "Focus on regulatory pathways and compliance requirements."
            elif agent.role == "Healthcare Economics Analyst":
                instructions = "Focus on financial impact and business case development."
            else:
                instructions = ""
            
            result = agent.analyze(topic, context, instructions)
            results.append(result)
            
            # Add to context for next agent
            if result.get("analysis"):
                context += f"\n\n{agent.role} Analysis:\n{result['analysis'][:500]}..."
        
        # Phase 2: Strategic Synthesis
        print(f"\n🔄 Final Phase: Strategic Synthesis")
        
        # Prepare comprehensive context for synthesizer
        synthesis_context = "\n\n".join([
            f"{r['agent']} Analysis:\n{r['analysis']}" 
            for r in results if r.get('analysis') and 'failed' not in r['analysis'].lower()
        ])
        
        synthesis_instructions = f"""Based on the analyses from domain, technical, regulatory, and economic experts, 
        create a comprehensive strategic analysis with:
        
        1. Executive Summary (key strategic insights)
        2. Integrated Analysis (how all factors interact)
        3. Strategic Recommendations (for different stakeholders)
        4. Implementation Roadmap (priorities and timeline)
        5. Risk Assessment (challenges and mitigation strategies)
        6. Future Outlook (strategic implications)
        
        Target: 1500-2000 words for executive decision-makers."""
        
        synthesis_result = self.agents[-1].analyze(topic, synthesis_context, synthesis_instructions)
        results.append(synthesis_result)
        
        # Calculate overall metrics
        total_duration = time.time() - start_time
        total_words = sum(r.get('word_count', 0) for r in results)
        successful_agents = sum(1 for r in results if r.get('analysis') and 'failed' not in r.get('analysis', '').lower())
        
        workflow_result = {
            "topic": topic,
            "analysis_type": analysis_type,
            "agents_used": len(self.agents),
            "successful_agents": successful_agents,
            "total_duration": total_duration,
            "total_words": total_words,
            "words_per_second": total_words / total_duration if total_duration > 0 else 0,
            "individual_results": results,
            "final_synthesis": synthesis_result.get('analysis', ''),
            "timestamp": datetime.now().isoformat()
        }
        
        self.workflow_history.append(workflow_result)
        
        print(f"\n⏱️ Analysis completed in {total_duration:.1f} seconds")
        print(f"📝 Generated {total_words:,} words ({total_words/total_duration:.1f} words/second)")
        print(f"✅ {successful_agents}/{len(self.agents)} agents completed successfully")
        print("=" * 70)
        
        return workflow_result
    
    def run_parallel_analysis(self, topic: str) -> Dict[str, Any]:
        """Run parallel analysis (simulated with sequential for demo)"""
        print("🔄 Running parallel analysis workflow...")
        # For simplicity, we'll run sequential but could implement async here
        return self.run_sequential_analysis(topic, "parallel")
    
    def run_longform_analysis(self, topic: str) -> Dict[str, Any]:
        """Run extended analysis for long-form content"""
        
        print(f"📄 Starting long-form content creation: {topic}")
        print("=" * 70)
        
        start_time = time.time()
        
        # Phase 1: Deep Research
        research_context = ""
        research_results = []
        
        for agent in [healthcare_expert, technical_analyst]:
            instructions = f"Conduct deep research on {topic}. Provide comprehensive background, current state, and detailed analysis. Target 600+ words."
            result = agent.analyze(f"Deep Research: {topic}", research_context, instructions)
            research_results.append(result)
            research_context += f"\n\n{result.get('analysis', '')}[:300]..."
        
        # Phase 2: Multi-Section Writing
        sections = []
        section_topics = [
            "Introduction and Background",
            "Current State and Technologies", 
            "Regulatory and Compliance Landscape",
            "Economic Impact and Business Case",
            "Implementation Strategies",
            "Future Outlook and Recommendations"
        ]
        
        full_context = "\n\n".join([r.get('analysis', '') for r in research_results])
        
        for i, section_topic in enumerate(section_topics):
            print(f"📝 Writing Section {i+1}: {section_topic}")
            
            instructions = f"""Write a comprehensive section on '{section_topic}' for the topic '{topic}'.
            
            Use the research context provided. Target 400-500 words for this section.
            Write in a professional, analytical style suitable for executive audiences."""
            
            result = strategic_synthesizer.analyze(f"{topic} - {section_topic}", full_context, instructions)
            sections.append(result)
        
        # Phase 3: Final Assembly
        print("🔗 Assembling final long-form document...")
        
        sections_context = "\n\n".join([
            f"Section: {section_topics[i]}\n{s.get('analysis', '')}" 
            for i, s in enumerate(sections)
        ])
        
        final_instructions = f"""Assemble the provided sections into a cohesive, comprehensive long-form document about '{topic}'.
        
        Create:
        1. Executive Summary (200 words)
        2. Smooth transitions between sections
        3. Comprehensive conclusion (300 words)
        4. Key recommendations summary
        
        Ensure consistent tone, eliminate redundancy, and create a publication-ready document of 2500+ words."""
        
        final_result = strategic_synthesizer.analyze(f"Final Assembly: {topic}", sections_context, final_instructions)
        
        # Calculate metrics
        total_duration = time.time() - start_time
        all_results = research_results + sections + [final_result]
        total_words = sum(r.get('word_count', 0) for r in all_results)
        
        longform_result = {
            "topic": topic,
            "analysis_type": "longform",
            "phases": ["research", "writing", "assembly"],
            "total_duration": total_duration,
            "total_words": total_words,
            "words_per_second": total_words / total_duration if total_duration > 0 else 0,
            "research_results": research_results,
            "section_results": sections,
            "final_document": final_result.get('analysis', ''),
            "timestamp": datetime.now().isoformat()
        }
        
        print(f"\n⏱️ Long-form creation completed in {total_duration:.1f} seconds")
        print(f"📄 Generated {total_words:,} words ({total_words/total_duration:.1f} words/second)")
        print("=" * 70)
        
        return longform_result

# Create orchestrator
orchestrator = MultiAgentOrchestrator(agents)
print("🎭 Multi-agent orchestrator ready!")

🎭 Multi-agent orchestrator ready!


In [5]:
# Cell 5: Execution Functions
def run_comprehensive_analysis(topic: str):
    """Run comprehensive 5-agent analysis"""
    return orchestrator.run_sequential_analysis(topic, "comprehensive")

def run_parallel_analysis(topic: str):
    """Run parallel analysis workflow"""
    return orchestrator.run_parallel_analysis(topic)

def run_longform_creation(topic: str):
    """Run long-form content creation"""
    return orchestrator.run_longform_analysis(topic)

def display_results(result: Dict[str, Any]):
    """Display results in a formatted way"""
    print("\n" + "=" * 60)
    print("📊 ANALYSIS RESULTS")
    print("=" * 60)
    
    print(f"Topic: {result['topic']}")
    print(f"Analysis Type: {result['analysis_type'].title()}")
    print(f"Duration: {result['total_duration']:.1f} seconds")
    print(f"Total Words: {result['total_words']:,}")
    print(f"Generation Speed: {result['words_per_second']:.1f} words/second")
    
    if 'successful_agents' in result:
        print(f"Agent Success Rate: {result['successful_agents']}/{result.get('agents_used', 0)}")
    
    print("\n" + "-" * 60)
    print("FINAL OUTPUT:")
    print("-" * 60)
    
    if 'final_synthesis' in result:
        print(result['final_synthesis'])
    elif 'final_document' in result:
        print(result['final_document'])
    else:
        print("No final output available")

print("🚀 Execution functions ready!")

🚀 Execution functions ready!


In [6]:
# Cell 6: Example Topics and Usage
complex_topics = [
    "AI-Driven Personalized Medicine: Multi-Domain Strategic Analysis",
    "Robotic Surgery Integration: Comprehensive Stakeholder Assessment", 
    "AI Diagnostic Imaging: Technical, Regulatory, and Economic Evaluation",
    "Telemedicine AI: Cross-Domain Implementation Strategy",
    "AI-Powered Drug Discovery: Feasibility and Market Analysis"
]

longform_topics = [
    "The Strategic Future of AI in Precision Medicine",
    "Comprehensive Guide to Healthcare AI Implementation",
    "AI Ethics in Medical Decision Making: Complete Framework",
    "Digital Health Transformation: Technology, Policy, and Economics"
]

print("🧩 Complex Analysis Topics:")
for i, topic in enumerate(complex_topics, 1):
    print(f"{i}. {topic}")

print("\n📄 Long-Form Content Topics:")
for i, topic in enumerate(longform_topics, 1):
    print(f"{i}. {topic}")

print("\n💡 Usage Examples:")
print("# Comprehensive 5-agent analysis")
print("result = run_comprehensive_analysis(complex_topics[0])")
print("display_results(result)")
print()
print("# Long-form content creation")  
print("longform_result = run_longform_creation(longform_topics[0])")
print("display_results(longform_result)")

🧩 Complex Analysis Topics:
1. AI-Driven Personalized Medicine: Multi-Domain Strategic Analysis
2. Robotic Surgery Integration: Comprehensive Stakeholder Assessment
3. AI Diagnostic Imaging: Technical, Regulatory, and Economic Evaluation
4. Telemedicine AI: Cross-Domain Implementation Strategy
5. AI-Powered Drug Discovery: Feasibility and Market Analysis

📄 Long-Form Content Topics:
1. The Strategic Future of AI in Precision Medicine
2. Comprehensive Guide to Healthcare AI Implementation
3. AI Ethics in Medical Decision Making: Complete Framework
4. Digital Health Transformation: Technology, Policy, and Economics

💡 Usage Examples:
# Comprehensive 5-agent analysis
result = run_comprehensive_analysis(complex_topics[0])
display_results(result)

# Long-form content creation
longform_result = run_longform_creation(longform_topics[0])
display_results(longform_result)


In [7]:
# Cell 7: Performance Comparison and Testing
def benchmark_workflows():
    """Benchmark different workflow types"""
    
    print("📊 WORKFLOW PERFORMANCE COMPARISON")
    print("=" * 50)
    
    # Simple test topic
    test_topic = "AI in Medical Imaging"
    
    print("\n🔸 LangChain Multi-Agent Workflows:")
    print(f"Test Topic: {test_topic}")
    print()
    print("Comprehensive Analysis (5 agents):")
    print("- Expected: 45-60 seconds")
    print("- Output: 1500-2000 words")
    print("- Agents: Healthcare → Technical → Regulatory → Economic → Synthesis")
    print()
    print("Long-Form Creation (6 phases):")
    print("- Expected: 60-90 seconds") 
    print("- Output: 2500+ words")
    print("- Phases: Research → Research → Write → Write → Write → Assemble")
    print()
    print("💡 Advantages of LangChain approach:")
    print("- More control over workflow logic")
    print("- Better error handling and recovery")
    print("- Easier to customize agent behavior")
    print("- Direct integration with other LangChain tools")
    print("- More transparent execution process")

benchmark_workflows()

📊 WORKFLOW PERFORMANCE COMPARISON

🔸 LangChain Multi-Agent Workflows:
Test Topic: AI in Medical Imaging

Comprehensive Analysis (5 agents):
- Expected: 45-60 seconds
- Output: 1500-2000 words
- Agents: Healthcare → Technical → Regulatory → Economic → Synthesis

Long-Form Creation (6 phases):
- Expected: 60-90 seconds
- Output: 2500+ words
- Phases: Research → Research → Write → Write → Write → Assemble

💡 Advantages of LangChain approach:
- More control over workflow logic
- Better error handling and recovery
- Easier to customize agent behavior
- Direct integration with other LangChain tools
- More transparent execution process


In [15]:
# Cell 8: Save and Export Functions
def save_analysis_result(result: Dict[str, Any], filename: str = None):
    """Save analysis results to file"""
    import os
    
    if not filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        safe_topic = result['topic'].lower().replace(' ', '_').replace(':', '').replace(',', '')[:50]
        filename = f"langchain_{result['analysis_type']}_{safe_topic}_{timestamp}.txt"
    
    # Ensure projects directory exists
    os.makedirs('/home/jovyan/projects', exist_ok=True)
    filepath = f'/home/jovyan/projects/{filename}'
    
    # Save content
    with open(filepath, 'w') as f:
        f.write(f"LANGCHAIN MULTI-AGENT ANALYSIS REPORT\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Topic: {result['topic']}\n")
        f.write(f"Analysis Type: {result['analysis_type'].title()}\n") 
        f.write(f"Generated: {result['timestamp']}\n")
        f.write(f"Duration: {result['total_duration']:.1f} seconds\n")
        f.write(f"Total Words: {result['total_words']:,}\n")
        f.write(f"Generation Speed: {result['words_per_second']:.1f} words/second\n")
        
        if 'successful_agents' in result:
            f.write(f"Agent Success Rate: {result['successful_agents']}/{result.get('agents_used', 0)}\n")
        
        f.write("\n" + "=" * 60 + "\n\n")
        
        # Write main content
        if 'final_synthesis' in result:
            f.write("STRATEGIC SYNTHESIS:\n")
            f.write("-" * 30 + "\n")
            f.write(result['final_synthesis'])
        elif 'final_document' in result:
            f.write("LONG-FORM DOCUMENT:\n")
            f.write("-" * 30 + "\n")
            f.write(result['final_document'])
        
        # Write individual agent results if available
        if 'individual_results' in result:
            f.write("\n\n" + "=" * 60 + "\n")
            f.write("INDIVIDUAL AGENT ANALYSES:\n")
            f.write("=" * 60 + "\n")
            
            for r in result['individual_results']:
                if r.get('analysis') and 'failed' not in r.get('analysis', '').lower():
                    f.write(f"\n{r['agent']}:\n")
                    f.write("-" * len(r['agent']) + "\n")
                    f.write(f"{r['analysis']}\n")
    
    print(f"✅ Analysis saved to: {filename}")
    return filepath

def create_workflow_summary():
    """Create summary of all workflow executions"""
    if not orchestrator.workflow_history:
        print("No workflow history available")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    filename = f"langchain_workflow_summary_{timestamp}.txt"
    filepath = f'/home/jovyan/projects/{filename}'
    
    with open(filepath, 'w') as f:
        f.write("LANGCHAIN MULTI-AGENT WORKFLOW SUMMARY\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total Workflows: {len(orchestrator.workflow_history)}\n\n")
        
        total_words = sum(w['total_words'] for w in orchestrator.workflow_history)
        total_time = sum(w['total_duration'] for w in orchestrator.workflow_history)
        avg_speed = total_words / total_time if total_time > 0 else 0
        
        f.write(f"Overall Statistics:\n")
        f.write(f"- Total Words Generated: {total_words:,}\n")
        f.write(f"- Total Time: {total_time:.1f} seconds\n")
        f.write(f"- Average Speed: {avg_speed:.1f} words/second\n\n")
        
        for i, workflow in enumerate(orchestrator.workflow_history, 1):
            f.write(f"Workflow {i}:\n")
            f.write(f"- Topic: {workflow['topic']}\n")
            f.write(f"- Type: {workflow['analysis_type']}\n")
            f.write(f"- Duration: {workflow['total_duration']:.1f}s\n")
            f.write(f"- Words: {workflow['total_words']:,}\n")
            f.write(f"- Speed: {workflow['words_per_second']:.1f} w/s\n")
            f.write("-" * 30 + "\n")
    
    print(f"📊 Workflow summary saved to: {filename}")
    return filepath

print("💾 Export functions ready!")
print("Use: save_analysis_result(result)")
print("Use: create_workflow_summary()")

💾 Export functions ready!
Use: save_analysis_result(result)
Use: create_workflow_summary()


In [8]:
# Cell 9: Test Execution
print("\n🧪 Ready to test LangChain multi-agent system!")
print("=" * 50)
print()
print("Quick test examples:")
print("1. test_result = run_comprehensive_analysis('AI in Medical Diagnostics')")
print("2. display_results(test_result)")
print("3. save_analysis_result(test_result)")
print()
print("🎯 LangChain Multi-Agent System Ready!")
print("Features:")
print("- ✅ Direct LangChain integration")
print("- ✅ Better error handling")
print("- ✅ Customizable workflows")
print("- ✅ Detailed progress tracking")
print("- ✅ Memory and context management")
print("- ✅ Performance optimization")


🧪 Ready to test LangChain multi-agent system!

Quick test examples:
1. test_result = run_comprehensive_analysis('AI in Medical Diagnostics')
2. display_results(test_result)
3. save_analysis_result(test_result)

🎯 LangChain Multi-Agent System Ready!
Features:
- ✅ Direct LangChain integration
- ✅ Better error handling
- ✅ Customizable workflows
- ✅ Detailed progress tracking
- ✅ Memory and context management
- ✅ Performance optimization


# This section will run everything and produce the results

In [9]:
# Comprehensive 5-agent analysis
result = run_comprehensive_analysis(complex_topics[0])
display_results(result)

# Long-form content creation
longform_result = run_longform_creation(longform_topics[0])
display_results(longform_result)

🚀 Starting comprehensive multi-agent analysis: AI-Driven Personalized Medicine: Multi-Domain Strategic Analysis

📋 Phase 1: Healthcare Domain Expert
✅ Healthcare Domain Expert completed analysis in 13.1s (486 words)

📋 Phase 2: AI Technical Analyst
✅ AI Technical Analyst completed analysis in 8.7s (444 words)

📋 Phase 3: Healthcare Regulatory Specialist
✅ Healthcare Regulatory Specialist completed analysis in 8.0s (433 words)

📋 Phase 4: Healthcare Economics Analyst
✅ Healthcare Economics Analyst completed analysis in 11.1s (610 words)

🔄 Final Phase: Strategic Synthesis
✅ Strategic Content Synthesizer completed analysis in 14.4s (586 words)

⏱️ Analysis completed in 55.3 seconds
📝 Generated 2,559 words (46.3 words/second)
✅ 5/5 agents completed successfully

📊 ANALYSIS RESULTS
Topic: AI-Driven Personalized Medicine: Multi-Domain Strategic Analysis
Analysis Type: Comprehensive
Duration: 55.3 seconds
Total Words: 2,559
Generation Speed: 46.3 words/second
Agent Success Rate: 5/5

-------

NameError: name 'save_analysis_result' is not defined

In [17]:
# Save results
#print(result)
save_analysis_result(result)
create_workflow_summary()

📊 Workflow summary saved to: langchain_workflow_summary_20250808_0417.txt


'/home/jovyan/projects/langchain_workflow_summary_20250808_0417.txt'